In [1]:
import torch
import torch.nn as nn
from torchvision import transforms
from torch.utils.data import DataLoader
from torch import optim

In [2]:
import sys
sys.path.insert(1, "../../../")

In [3]:
from train import *
from data_preprocessing import *
from Models.yolov8cls_path import Model

In [4]:
if torch.cuda.is_available():
    device=torch.device('cuda')
else:
    device=torch.device('cpu') 

In [5]:
model = Model(num_classes=10, 
              residual_connection=True, 
              CSP=True, 
              add_hidden=True,
              classifyV8=False,
              bottleneck=1.0, 
              variant='s', 
              device=device, 
              dtype=torch.float32)

In [6]:
data_path = '../../../../dummy_datasets/'
norms_path = os.path.join(data_path, 'norms.json')

In [7]:
means = get_means(path=norms_path, train_loader=None)
stds = get_stds(path=norms_path, train_loader=None)

Means are: [0.4405549168586731, 0.4407285749912262, 0.4381718039512634]
stds are: [0.25142669677734375, 0.25270089507102966, 0.25131651759147644]


In [8]:
transformations = transforms.Compose([transforms.ToTensor(),
                                      transforms.Resize((224, 224)),
                                      transforms.Normalize(mean=means, std=stds)])

In [9]:
train_dataset = ImageNetSubset(path=data_path, train=True, transform=transformations, half=False, show=False)
val_dataset = ImageNetSubset(path=data_path, train=False, transform=transformations, half=False, show=False)

In [10]:
epochs=3
train_loader = DataLoader(train_dataset, batch_size=256, shuffle=True, num_workers=4)
val_loader = DataLoader(val_dataset, batch_size=256, shuffle=False, num_workers=4)

In [11]:
optimizer = optim.SGD(model.parameters(), lr=0.001, momentum=0.9, weight_decay=0.0001)

In [12]:
loss_fn = nn.NLLLoss()

In [13]:
prof = torch.profiler.profile(
        schedule=torch.profiler.schedule(wait=1, warmup=1, active=epochs-2, repeat=1),
        on_trace_ready=torch.profiler.tensorboard_trace_handler('../../log/YOLOv8cls-version-4/training'),
        record_shapes=True,
        profile_memory=True,
        with_flops=True,
        with_modules=True,
        with_stack=True)


Profiling your personal module 
https://pytorch.org/tutorials/beginner/profiler.html

In [14]:
history, gradient_stats, prof = train(epochs, train_loader, val_loader, model, optimizer, loss_fn, prof, model_path='../../log/YOLOv8cls-version-4/training/models/')

2024-11-18 18:51:28.584639 Epoch 1 
2024-11-18 18:51:35.341753 Batch 1 
[Train] Accuracy: 3.3333%, Loss per batch: 2.3261
2024-11-18 18:51:46.827737 Batch 1 
[Val] Accuracy: 10.0%, loss per batch: 2.3026
2024-11-18 18:51:48.193661 Epoch 2 
2024-11-18 18:51:54.571732 Batch 1 
[Train] Accuracy: 6.6667%, Loss per batch: 2.3049
2024-11-18 18:52:02.892170 Batch 1 
[Val] Accuracy: 10.0%, loss per batch: 2.3026
2024-11-18 18:52:05.643868 Epoch 3 
2024-11-18 18:52:12.166517 Batch 1 
[Train] Accuracy: 20.0%, Loss per batch: 2.2679
2024-11-18 18:52:20.761434 Batch 1 


KeyboardInterrupt: 

In [ ]:
print(prof.key_averages().table(row_limit=-1))

In [1]:
save_train_outputs('../../log/YOLOv8cls-version-4/training', history, gradient_stats)

NameError: name 'save_train_outputs' is not defined